<a href="https://colab.research.google.com/github/AtogmatomaProgramming/OdonScanAI/blob/main/manage_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#=== En caso de que no esté instalado scikit-learn
!pip install scikit-learn
!pip install tensorflow

In [ ]:
#=== Importar librerias ===

import os
import json
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#=== Funciones de trabajo ===

# Función para obtener rutas de imágenes y etiquetas
def obtener_imagenes_y_etiquetas(carpeta_base, archivo_json):
    # Cargar o inicializar el JSON
    if os.path.exists(archivo_json):
        with open(archivo_json, 'r') as f:
            especies_dict = json.load(f)
    else:
        especies_dict = {}

    image_paths = []
    labels = []

    for especie in sorted(os.listdir(carpeta_base)):
        especie_dir = os.path.join(carpeta_base, especie)
        if os.path.isdir(especie_dir):
            if especie not in especies_dict:
                especies_dict[especie] = len(especies_dict)
            label = especies_dict[especie]

            for root, _, files in os.walk(especie_dir):
                for file in files:
                    if file.endswith(('.png', '.jpg', '.jpeg')):
                        image_paths.append(os.path.join(root, file))
                        labels.append(label)

    # Guardar el JSON actualizado
    with open(archivo_json, 'w') as f:
        json.dump(especies_dict, f)

    return image_paths, labels

# Función para preparar el dataset utilizando tf.data
def preparar_datasets(image_paths, labels, batch_size=32, validation_split=0.2):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Cargar y preprocesar las imágenes
    def cargar_y_preprocesar(ruta, etiqueta):
        imagen = tf.io.read_file(ruta)
        imagen = tf.image.decode_jpeg(imagen, channels=3)
        imagen = tf.image.resize(imagen, [224, 224])
        imagen = tf.image.convert_image_dtype(imagen, tf.float32)  # Normalizar
        return imagen, etiqueta

    dataset = dataset.map(cargar_y_preprocesar, num_parallel_calls=tf.data.AUTOTUNE)

    # Dividir en entrenamiento y validación
    dataset_size = len(image_paths)
    train_size = int((1 - validation_split) * dataset_size)

    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size)

    # Configurar el dataset para batching, mezclado y cache
    train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds

# Definir el modelo
def crear_modelo(num_clases):
    modelo = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_clases, activation='softmax')
    ])

    modelo.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
    return modelo

# Función principal
def main(carpeta_base, archivo_json, model_name):
    # Crear rutas de imágenes y etiquetas desde las carpetas de clases
    image_paths, labels = obtener_imagenes_y_etiquetas(carpeta_base, archivo_json)

    # Preparar los datasets de entrenamiento y validación
    train_ds, val_ds = preparar_datasets(image_paths, labels)

    # Crear el modelo con el número de clases detectado
    modelo = crear_modelo(num_clases=len(set(labels)))

    # Entrenar el modelo
    history = modelo.fit(train_ds, validation_data=val_ds, epochs=10)

    # Guardar el modelo entrenado
    modelo.save(model_name)



In [ ]:
# Llamada a la función main con la ruta de carpeta base y el archivo JSON

carpeta_base = "/content/drive/My Drive/alpha_training/frames"
archivo_json= "/content/drive/My Drive/alpha_training/etiquetas.json"
model_name = "/content/drive/My Drive/alpha_training/modelo_gamma.h5"

main(carpeta_base, archivo_json, model_name)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 725s 4s/step - accuracy: 0.7462 - loss: 74.4578 - val_accuracy: 0.0580 - val_loss: 7.2932
Epoch 2/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 705s 4s/step - accuracy: 0.8372 - loss: 0.9831 - val_accuracy: 0.0727 - val_loss: 23.4851
Epoch 3/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 703s 4s/step - accuracy: 0.9382 - loss: 0.2373 - val_accuracy: 0.0816 - val_loss: 23.9898
Epoch 4/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 708s 4s/step - accuracy: 0.9919 - loss: 0.0324 - val_accuracy: 0.0727 - val_loss: 37.9359
Epoch 5/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 704s 4s/step - accuracy: 0.9956 - loss: 0.0170 - val_accuracy: 0.0617 - val_loss: 40.8363
Epoch 6/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 685s 4s/step - accuracy: 0.9969 - loss: 0.0160 - val_accuracy: 0.0683 - val_loss: 35.8833
Epoch 7/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 756s 4s/step - accuracy: 0.9984 - loss: 0.0073 - val_accuracy: 0.0794 - val_loss: 33.5463
Epoch 8/10
171/171 ━━━━━━━━━━━━━━━━━━━━ 726s 4s/step - accuracy: 0.9738 - loss: 0.3517 - v

In [ ]:
#=== Aplicar funciones ===

# Montar el acceso a drive

# drive.mount('/content/drive')

# Ruta de la carpeta donde están las especies (ajusta esta ruta)
# carpeta_base = "/content/drive/My Drive/alpha_training/frames"
# archivo_json= "/content/drive/My Drive/alpha_training/etiquetas.json"

# Actualizar el archivo JSON con las especies y sus etiquetas
# main(carpeta_base, archivo_json)